In [60]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime
from datetime import date,timedelta



def find_crypto_information(soup):
    # get this to work for all cryptocurrencies for future use. 
    data = []    
    date_str = soup.find('h1').text.strip('\n').split('-')
    date = datetime.strptime(date_str[1], ' %B %d, %Y')

    table = soup.find('table', id='currencies-all')
    rows = table.find_all('tbody')   
    for row in rows:
        try:
            symbol = row.find('td', class_='text-left col-symbol').text
            price = row.find('a', class_='price').text
        except AttributeError:
            print('error')
            continue

        data.append([symbol, price, date])
        
    return data

#Only stores historical links
def get_urls(soup):
    urls = []
    url_base = 'https://coinmarketcap.com' 
    for link in soup.find_all('a',href=True):
        link_text = link['href']
        if fuzz.partial_ratio('/historical/201',link_text) == 100 and \
            fuzz.partial_ratio('/201',link_text) == 100 and link_text!='/':
            urls.append(url_base + link_text)
    return urls

r = requests.get('https://coinmarketcap.com/historical/')
soup = BeautifulSoup(r.text, 'html5lib')    
urls = get_urls(soup)



In [59]:
#Now get the historical prices
new_urls = []
for url in urls:
    tmp = url.split('/')
    date = datetime.strptime(tmp[-2],'%Y%m%d')
    
    end_date = date.today()
    start_date = end_date - timedelta(days=370)
    if date >= start_date:
        new_urls.append(url)
        

In [65]:
price_data = []

for url in new_urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html5lib')
    output = find_crypto_information(soup)
    print(output)

    price_data.append(output)
        
       
    

[['BTC', '$2582.68', datetime.datetime(2017, 7, 9, 0, 0)]]
[['BTC', '$1995.52', datetime.datetime(2017, 7, 16, 0, 0)]]
[['BTC', '$2763.20', datetime.datetime(2017, 7, 23, 0, 0)]]
[['BTC', '$2689.44', datetime.datetime(2017, 7, 30, 0, 0)]]
[['BTC', '$3214.71', datetime.datetime(2017, 8, 6, 0, 0)]]
[['BTC', '$4071.58', datetime.datetime(2017, 8, 13, 0, 0)]]
[['BTC', '$4131.82', datetime.datetime(2017, 8, 20, 0, 0)]]
[['BTC', '$4358.81', datetime.datetime(2017, 8, 27, 0, 0)]]
[['BTC', '$4632.25', datetime.datetime(2017, 9, 3, 0, 0)]]
[['BTC', '$4083.70', datetime.datetime(2017, 9, 10, 0, 0)]]
[['BTC', '$3536.84', datetime.datetime(2017, 9, 17, 0, 0)]]
[['BTC', '$3725.65', datetime.datetime(2017, 9, 24, 0, 0)]]
[['BTC', '$4318.34', datetime.datetime(2017, 10, 1, 0, 0)]]
[['BTC', '$4438.85', datetime.datetime(2017, 10, 8, 0, 0)]]
[['BTC', '$5729.63', datetime.datetime(2017, 10, 15, 0, 0)]]
[['BTC', '$5933.34', datetime.datetime(2017, 10, 22, 0, 0)]]
[['BTC', '$5779.54', datetime.datetime(20

In [85]:
store_price_data = price_data

In [89]:
df = pd.DataFrame(store_price_data)
df.to_pickle('btc_price.pkl')